In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/imdb-ptbr/imdb-reviews-pt-br.csv


# Imports do Projeto

In [2]:
import torch
print(torch.cuda.is_available())

True


In [3]:
import os
import pandas as pd
import numpy as np
import torch
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
import time
import re
from transformers import (
    DistilBertTokenizer,  # Tokenizador específico do DistilBERT
    DistilBertForSequenceClassification,  # Modelo para classificação
    get_linear_schedule_with_warmup
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Pré-processamento dos dados

In [4]:
# Definiçao de semente aleatoria para consistencia
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

In [5]:
# Verificar disponibilidade de GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Usando dispositivo: {device}")

Usando dispositivo: cuda


In [6]:
reviews_train_df = pd.read_csv('/kaggle/input/imdb-ptbr/imdb-reviews-pt-br.csv')
reviews_train_df.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [7]:
# Converter os valores da coluna 'sentiment' para binário
reviews_train_df['sentiment'] = reviews_train_df['sentiment'].map({'neg': 0, 'pos': 1})

In [8]:
reviews_train_df.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",0
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,0
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",0
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,0
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,0


In [9]:
# Ajustando df para colunas que importam
reviews_train_df_novo = reviews_train_df[['text_pt', 'sentiment']]
reviews_train_df_novo.head()

,text_pt,sentiment
0,"Mais uma vez, o Sr. Costner arrumou um filme p...",0
1,Este é um exemplo do motivo pelo qual a maiori...,0
2,"Primeiro de tudo eu odeio esses raps imbecis, ...",0
3,Nem mesmo os Beatles puderam escrever músicas ...,0
4,Filmes de fotos de latão não é uma palavra apr...,0


In [10]:
# Remoçao de palavras sem sentido semantico utilizando o spaCy, para manter o contexto

!pip install spacy
!python -m spacy download pt_core_news_sm

import spacy

nlp = spacy.load("pt_core_news_sm")

def preprocess_text(text):
    text = text.lower()

    doc = nlp(text)

    cleaned_tokens = [
        token.lemma_ for token in doc
        if not token.is_punct
        and not token.is_space
        and not token.is_stop
        and not token.like_url
        and not token.like_email
        and token.is_alpha
        and len(token.text) > 2
    ]

    cleaned_text = " ".join(cleaned_tokens)
    return cleaned_text

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 83.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [11]:
reviews_train_df_novo.loc[:, 'text_pt_processed'] = reviews_train_df_novo['text_pt'].apply(preprocess_text)

<ipython-input-11-9c78f5dd7f79>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  reviews_train_df_novo.loc[:, 'text_pt_processed'] = reviews_train_df_novo['text_pt'].apply(preprocess_text)


In [12]:
reviews_train_df_novo.head()

,text_pt,sentiment,text_pt_processed
0,"Mais uma vez, o Sr. Costner arrumou um filme p...",0,costner arrumar filme necessário terrível seqü...
1,Este é um exemplo do motivo pelo qual a maiori...,0,motivo filme ação mesmo genérico chato valha p...
2,"Primeiro de tudo eu odeio esses raps imbecis, ...",0,odeiar raps imbecis poder agir tiver arma pres...
3,Nem mesmo os Beatles puderam escrever músicas ...,0,beatle escrever música gostar walter hill ser ...
4,Filmes de fotos de latão não é uma palavra apr...,0,filme foto latão palavra apropriar verdade ous...


In [13]:
reviews_train_df_novo = reviews_train_df_novo.drop('text_pt', axis=1)

In [14]:
# Ajustando ordens da coluna (por puro toc)
reviews_train_df_novo = reviews_train_df_novo[['text_pt_processed', 'sentiment']]
reviews_train_df_novo.head()

,text_pt_processed,sentiment
0,costner arrumar filme necessário terrível seqü...,0
1,motivo filme ação mesmo genérico chato valha p...,0
2,odeiar raps imbecis poder agir tiver arma pres...,0
3,beatle escrever música gostar walter hill ser ...,0
4,filme foto latão palavra apropriar verdade ous...,0


# Treinos e testes

In [15]:
#Tokenizacao do dataset
model_name = "adalbertojunior/distilbert-portuguese-cased"  # DistilBERT em português
tokenizer = DistilBertTokenizer.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/513 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/210k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DistilBertTokenizer'.


In [16]:
# Separaçao dos dados de treinos e testes
train_texts, val_texts, train_labels, val_labels = train_test_split(
    reviews_train_df_novo['text_pt_processed'].values,
    reviews_train_df_novo['sentiment'].values,
    test_size=0.2,
    random_state=SEED
    )

In [17]:
# Conversão para lista de Strings
train_texts = [str(text) for text in train_texts]  # Garante que são strings
val_texts = [str(text) for text in val_texts]

In [18]:
print(type(train_texts))
print(type(train_labels))

<class 'list'>
<class 'numpy.ndarray'>


In [19]:
# classe para criaçao de um Dataset PyTorch permite otimizar o carregamento de dados e paralelizar a computação

class TextClassificationDataset(Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=256):
        self.encodings = tokenizer(
            texts, padding=True, truncation=True, max_length=max_length, return_tensors="pt"
        )
        self.labels = torch.tensor(labels)

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return {
            "input_ids": self.encodings["input_ids"][idx],
            "attention_mask": self.encodings["attention_mask"][idx],
            "labels": self.labels[idx],
        }

In [20]:
#Criaçao de dataset para treino e validaçao.
train_dataset = TextClassificationDataset(train_texts, train_labels, tokenizer)
val_dataset = TextClassificationDataset(val_texts, val_labels, tokenizer)

In [21]:
# Criacao de dataLoader para alimentar os modelos em lotes. O DataLoader permite carregar os dados em lotes (batch_size), tornando o treinamento mais eficiente
batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [22]:
# Carregamento do modelo pré-treinado BERT para classificação
num_labels = len(set(train_labels))  # Número de classes (geralmente 2 para sentimentos: positivo/negativo)
model = DistilBertForSequenceClassification.from_pretrained(
    model_name,
    num_labels=num_labels,
    attention_probs_dropout_prob=0.3,  # Recomendado para evitar overfitting
    hidden_dropout_prob=0.3
)
model.to(device)  # Mover para GPU se disponível

You are using a model of type bert to instantiate a model of type distilbert. This is not supported for all configurations of models and can yield errors.


model.safetensors:   0%|          | 0.00/266M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at adalbertojunior/distilbert-portuguese-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight', 'transformer.layer.0.attention.k_lin.bias', 'transformer.layer.0.attention.k_lin.weight', 'transformer.layer.0.attention.out_lin.bias', 'transformer.layer.0.attention.out_lin.weight', 'transformer.layer.0.attention.q_lin.bias', 'transformer.layer.0.attention.q_lin.weight', 'transformer.layer.0.attention.v_lin.bias', 'transformer.layer.0.attention.v_lin.weight', 'transformer.layer.0.ffn.lin1.bias', 'transformer.layer.0.ffn.lin1.weight', 'transformer.layer.0.ffn.lin2.bias', 'transformer.layer.0.ffn.lin2.weight', 'transformer.layer.0.output_layer_norm.bias', 'transformer.layer.0.output_layer_norm.weight', 'transformer.layer.0.sa_layer_norm.bias', 'transformer.layer.0.sa_layer_norm.weight', 'transformer.layer.1.attention.k_lin.bias'

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(29794, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [23]:
# Função de perda para classificação
criterion = torch.nn.CrossEntropyLoss()

# Otimizador AdamW recomendado para transformers
optimizer = AdamW(model.parameters(), lr=2e-5, eps=1e-8)

In [24]:
from torch.cuda.amp import autocast, GradScaler

# Definindo épocas e Early stop
num_epochs = 5
best_val_loss = float("inf")
patience = 2

def train_loop_fn(loader, model, optimizer, device):
    scaler = GradScaler()  # Instância do GradScaler ✅
    best_val_loss = float("inf")
    patience_counter = 0

    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        
        for batch in loader:
            optimizer.zero_grad()  # Resetar gradientes antes de cada passo
            
            inputs = {k: v.to(device) for k, v in batch.items()}
            
            # Forward pass com autocast para mixed precision 💡
            with autocast():
                outputs = model(**inputs)
                loss = outputs.loss
            
            # Backpropagation com scaling de gradientes 
            scaler.scale(loss).backward()  # ⚠️ Não usar loss.backward() 
            
            # Atualização dos pesos
            scaler.step(optimizer)  # Substitui optimizer.step()
            scaler.update()  # Ajusta o scale factor para o próximo passo
            
            total_loss += loss.item()

        avg_loss = total_loss / len(loader)
        print(f"🔹 Época {epoch+1}: Loss médio: {avg_loss:.4f}")

        # Validação
        model.eval()
        total_val_loss = 0
        predictions, true_labels = [], []
        
        with torch.no_grad():
            for batch in val_loader:
                inputs = {k: v.to(device) for k, v in batch.items()}
                outputs = model(**inputs)
                total_val_loss += outputs.loss.item()
                
                preds = torch.argmax(outputs.logits, dim=1)
                predictions.extend(preds.cpu().numpy())
                true_labels.extend(inputs['labels'].cpu().numpy())

        avg_val_loss = total_val_loss / len(val_loader)
        accuracy = accuracy_score(true_labels, predictions)
        print(f"🔸 Época {epoch+1}: Loss Validação: {avg_val_loss:.4f}, Acurácia: {accuracy:.4f}")

        # Early Stopping
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            # Salvar melhor modelo
            torch.save(model.state_dict(), "/kaggle/working/melhor_modelo.pt")
        else:
            patience_counter += 1
            print(f"⚠️ EarlyStopping: {patience_counter}/{patience}")

        if patience_counter >= patience:
            print("⏹️ EarlyStopping ativado!")
            break

In [25]:
# Execução do treinamento
train_loop_fn(train_loader, model, optimizer, device)

<ipython-input-24-130954210f36>:9: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()  # Instância do GradScaler ✅
<ipython-input-24-130954210f36>:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


🔹 Época 1: Loss médio: 0.4019
🔸 Época 1: Loss Validação: 0.3311, Acurácia: 0.8621


<ipython-input-24-130954210f36>:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


🔹 Época 2: Loss médio: 0.3001
🔸 Época 2: Loss Validação: 0.3189, Acurácia: 0.8691


<ipython-input-24-130954210f36>:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


🔹 Época 3: Loss médio: 0.2591
🔸 Época 3: Loss Validação: 0.3342, Acurácia: 0.8702
⚠️ EarlyStopping: 1/2


<ipython-input-24-130954210f36>:23: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():


🔹 Época 4: Loss médio: 0.2311
🔸 Época 4: Loss Validação: 0.3503, Acurácia: 0.8668
⚠️ EarlyStopping: 2/2
⏹️ EarlyStopping ativado!


# Salvando Modelo

In [26]:
# Salvar modelo final (após todo o treinamento)
model.save_pretrained("/kaggle/working/imdb-ptbrmodelo_bert_treinado")
tokenizer.save_pretrained("/kaggle/working/imdb-ptbrmodelo_bert_treinado")

('/kaggle/working/imdb-ptbrmodelo_bert_treinado/tokenizer_config.json',
 '/kaggle/working/imdb-ptbrmodelo_bert_treinado/special_tokens_map.json',
 '/kaggle/working/imdb-ptbrmodelo_bert_treinado/vocab.txt',
 '/kaggle/working/imdb-ptbrmodelo_bert_treinado/added_tokens.json')